In [1]:
from conll_iterator import ConllIterator
iterator = ConllIterator('sample/sample_corpus.conllu', ['form', 'upos'], mode='sentences', join_char='/')
iterator.sample(10)

['Regolamento/NOUN', '(/PUNCT', 'CE/PROPN', ')/PUNCT', 'n/NOUN', './PUNCT', '15/2008/NUM', 'di/ADP', 'il/DET', 'Consiglio/NOUN', 'di/ADP', 'il/DET', '20/NUM', 'dicembre/NOUN', '2007/NUM', 'che/PRON', 'modifica/VERB', 'il/DET', 'regolamento/NOUN', '(/PUNCT', 'CE/PROPN', ')/PUNCT', 'n./NOUN', '2100/94/NUM', 'per/ADP', 'quanto/PRON', 'riguarda/VERB', 'la/DET', 'legittimazione/NOUN', 'a/ADP', 'presentare/VERB', 'domanda/NOUN', 'di/ADP', 'privativa/NOUN', 'comunitaria/ADJ', 'per/ADP', 'ritrovati/NOUN', 'vegetali/ADJ']
['visto/VERB', 'il/DET', 'trattato/NOUN', 'che/PRON', 'istituisce/VERB', 'la/DET', 'Comunità/NOUN', 'europea/ADJ', ',/PUNCT', 'in/ADP', 'particolare/NOUN', 'l’/DET', 'articolo/NOUN', '308/NUM', ',/PUNCT']
['vista/VERB', 'la/DET', 'proposta/NOUN', 'di/ADP', 'la/DET', 'Commissione/NOUN', ',/PUNCT']
['visto/VERB', 'il/DET', 'parere/NOUN', 'di/ADP', 'il/DET', 'Parlamento/NOUN', 'europeo/ADJ', ',/PUNCT']
['(/PUNCT', '1/NUM', ')/PUNCT', 'Il/DET', 'regolamento/NOUN', '(/PUNCT', 'CE/P

Example usage 1: Training word2vec with lemma+POS

In [ ]:
%pip install gensim

In [150]:
from gensim.models import Word2Vec
from tqdm import tqdm
from conll_iterator import ConllIterator

sentences = ConllIterator('sample/sample_corpus.conllu', fields=['lemma', 'upos'], mode='sentences', join_char='/')
w2v_parameters = {'vector_size': 25, 'window': 5, 'min_count': 1, 'sg': 1, 'epochs': 15}
model = Word2Vec(tqdm(sentences), workers=5, **w2v_parameters)
model.save('sample/sample_w2v')
word_vectors = model.wv
similar = list(zip(*word_vectors.most_similar('Pecorino/PROPN')[:10]))[0]
print("Most similar words to Pecorino/PROPN:")
print(similar)

100%|██████████| 1807/1807 [00:00<00:00, 5863.54it/s]


Most similar words to Pecorino/PROPN:
('Romano/PROPN', 'Spressa/PROPN', 'Chianti/PROPN', 'Classico/PROPN', 'Grana/PROPN', 'Giudicarie/PROPN', 'Padano/PROPN', 'Lenteja/PROPN', 'Queso/PROPN', 'Cappero/PROPN')


Example usage 2: Keyword extraction via tf-idf

In [112]:
from itertools import chain
from collections import Counter

docs = ConllIterator('sample/sample_corpus.conllu', fields=['lemma', 'upos'], lower=['lemma'], mode='documents')
doc_tf = list()
df = Counter()
allowed_pos = ['NOUN', 'PROPN','VERB', 'ADJ']
for d in docs:
    tokens = list(chain(*d))
    tokens = [t[0] for t in tokens if t[1] in allowed_pos]
    tf = Counter(tokens)
    df.update(set(tokens))
    doc_tf.append(tf)

doc_keywords = list()
for d in doc_tf:
    doc_tfidf = [(w, d[w]/df[w]) for w in d]
    doc_tfidf = sorted(doc_tfidf, key=lambda x:x[1], reverse=True)
    doc_keywords.append(list(zip(*doc_tfidf[:10]))[0])

for i, k in enumerate(doc_keywords[:20]):
    print('keywords of doc {}:'.format(i+1), '; '.join(k))

keywords of doc 1: privativa; ritrovato; legittimazione; persona; assimilare; vegetale; giuridico; comunitario; valido; facilitare
keywords of doc 2: albo; espressione; farmfresh; turkey; spagnolo; lingua; leche; granja; attestazione; serrano
keywords of doc 3: reciproco; repubblica; romania; negoziato; protocollo; bulgaria; marocco; associazione; contingente; aspetto
keywords of doc 4: gögginger; bier; cancellazione; cancellare; precedere; germania; denominazione; regolamento; richiesta; luce
keywords of doc 5: valtellina; mela; denominazione; regolamento; marzo; italia; unione; europea; ce; numero
keywords of doc 6: rogal; świętomarciński; polonia; regolamento; denominazione; ce; ottobre; numero; articolo; protetto
keywords of doc 7: φάβα; σαντορίνης; fava; santorinis; grecia; denominazione; regolamento; dop; ottobre; unione
keywords of doc 8: kalix; löjrom; svezia; denominazione; regolamento; novembre; dop; unione; europea; ce
keywords of doc 9: coliflor; calahorra; ritirare; regola